In [2]:
import xarray as xr

In [3]:
# Thin netCDF
# Dataset: NUCAPS
# Open file, extract all variables
indir='input/'
odir='output/'

fname = "NUCAPS-EDR_v2r0_j01_s20190425175959_e20190425180557_c20190425180557"
nucaps = xr.open_dataset(indir+fname+'.nc')
tmp = list(nucaps.variables)

In [4]:
# List vars to leave in file
keeps = ['CO', 'Latitude', 'Longitude']

In [5]:
# Remove the "keep" variables from the full var list
encoding_dict = {}
for keep in keeps:
    tmp.remove(keep)
    dict_entry = { keep : {"dtype" : "float32", "zlib" : True, '_FillValue': -999.9} }
    encoding_dict.update(dict_entry)

In [6]:
# remove everything else
nucaps_thinned = nucaps.drop(tmp)

In [7]:
nucaps_thinned.to_netcdf(odir+fname+"_thinned.nc", engine="netcdf4", encoding=encoding_dict)

In [8]:
nucaps.close()

In [4]:
# Thin HDF
# Dataset: IMERG

In [12]:
fname = '3B-HHR.MS.MRG.3IMERG.20170827-S120000-E122959.0720.V06B'
grp_name="Grid/"

imerg = xr.open_dataset(indir+fname+".HDF5", group = grp_name)
tmp = list(imerg.variables)

In [13]:
keeps = ['precipitationCal', 'lon', 'lat', 'time', 'precipitationQualityIndex' ]

encoding_dict = {}
for keep in keeps:
    tmp.remove(keep)
    dict_entry = { keep : {"dtype": "float32", "zlib": True, '_FillValue': -9999.9} }
    encoding_dict.update(dict_entry)

In [14]:
imerg_thinned = imerg.drop(tmp)

In [15]:
coverage = [-94.3, 28.9, -88.8, 33.1]
imerg_thinned_small = imerg_thinned.where(
    (imerg_thinned.lat > 28) & (imerg_thinned.lat < 33) & (imerg_thinned.lon > -94) & (imerg_thinned.lon < -88), drop=True)

In [16]:
imerg_thinned_small

<xarray.Dataset>
Dimensions:                    (time: 1, lon: 60, lat: 50)
Coordinates:
  * time                       (time) object 2017-08-27 12:00:00
  * lon                        (lon) float32 -93.95 -93.85 ... -88.15 -88.05
  * lat                        (lat) float32 28.05 28.15 28.25 ... 32.85 32.95
Data variables:
    precipitationCal           (time, lon, lat) float32 2.111 2.374 ... 0.01872
    precipitationQualityIndex  (time, lon, lat) float32 0.677 0.677 ... 0.3976
Attributes:
    GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegistration=CENTER;\nLatitudeRe...

In [17]:
imerg_thinned_small.to_netcdf(odir+fname+"_thinned.nc", engine="netcdf4", encoding=encoding_dict)